In [1]:
import os
import sys
from copy import deepcopy

import matplotlib.pyplot as plt
import networkx as nx

if os.path.abspath('..') not in sys.path:
    sys.path.append(os.path.abspath('..'))
if os.path.abspath('../../real_data_experiments') not in sys.path:
    sys.path.append(os.path.abspath('../../real_data_experiments'))
from Main import plot_multiplex_with_ap_removal
import pymnet

# Preliminaries

In [2]:
def find_component(node, connected_components):
    for i, component in enumerate(connected_components):
        if node in component:
            return i
    raise ValueError(f'Component for node {node} not found')

In [3]:
def cascade_remove_links(G):

    keep_going = True
    while keep_going:
        keep_going = False

        clusters_in_layers = [list(nx.connected_components(g)) for g in G]

        for i, g in enumerate(G):
            clusters_to_search = clusters_in_layers[:i] + clusters_in_layers[i + 1:]
            for edge in g.edges():
                if any(find_component(edge[0], clusters) != find_component(edge[1], clusters)
                       for clusters in clusters_to_search):
                    keep_going = True
                    g.remove_edge(*edge)
    
    return G

In [4]:
def articulation_points_multiplex(G):
    ap = set(nx.articulation_points(G[0]))
    return ap.union(*[nx.articulation_points(g) for g in G[1:]])

In [5]:
def max_comp(g):
    comps = list(nx.connected_components(g))
    if len(comps) == 0:
        return []
    return max(comps, key=len)

In [6]:
def draw_two_layer(G_orig, G, layer_labels, show_labels, pos, ax):
    """Supports only one or two layers
    """
    onet = pymnet.MultiplexNetwork(couplings='categorical')
    for label in layer_labels:
        onet.add_layer(label)
    L = len(layer_labels)

    GRAY = '#84817a'
    BLACK = '#485460'

    art_points = articulation_points_multiplex(G)
    removed = [n for n in G_orig[0] if n not in G[0]]
    node_colors = {(node, layer): 'r'
                   for layer in layer_labels
                   for node in art_points}
    node_colors.update({(node, layer): GRAY
                        for layer in layer_labels
                        for node in removed})
    edge_colors = {}
    edge_alpha = {}
    edge_width = {}

    for n in G_orig[0].nodes:
        onet.add_node(n)
        if L == 2:
            edge_width[((n, layer_labels[0]), (n, layer_labels[1]))] = .2

    for label, g in zip(layer_labels, G_orig):
        for e1, e2 in g.edges:
            onet[e1, e2, label] = 1
            if e1 in removed or e2 in removed:
                # edge_colors[((e1, 'Facebook'), (e2, 'Facebook'))] = GRAY
                edge_alpha[((e1, label), (e2, label))] = .3
                # edge_width[((e1, 'Facebook'), (e2, 'Facebook'))] = .2
            else:
                # edge_width[((e1, 'Facebook'), (e2, 'Facebook'))] = .5
                edge_colors[((e1, label), (e2, label))] = BLACK

    return pymnet.draw(
        onet,
        elev=20,
        layergap=1.35,
        camera_dist=10,
        figsize=(24, 8),
        nodeCoords=pos,
        ax=ax,
        defaultLayerColor='#e8f8f8',
        # defaultLayerColor='#81ecec',
        # defaultLayerColor='#dff9fb',
        layerLabelDict={} if show_labels else {lbl: '' for lbl in layer_labels},
        defaultLayerLabelLoc=(.7, 1),
        defaultLayerLabelSize=20,
        nodeLabelRule={},
        nodeColorDict=node_colors,
        edgeColorDict=edge_colors,
        # edgeWidthDict=edge_width,
        defaultEdgeWidth=.1,
        edgeAlphaDict=edge_alpha,
    )

In [7]:
def draw_process(G, pos, layer_labels, figtitle):
    plt.close('all')
    G_orig = deepcopy(G)
    ap_steps = []

    while True:
        G = cascade_remove_links(G)
        gcc = max_comp(G[0])
        for g in G:
            g.remove_nodes_from([n for n in g if n not in gcc])

        ap_steps.append(deepcopy(G))

        art_points = articulation_points_multiplex(G)
        for g in G:
            g.remove_nodes_from(art_points)
        if not art_points:
            break

    fig, ax = plt.subplots(nrows=1, ncols=len(ap_steps), squeeze=False,
                           subplot_kw=dict(projection='3d'),
                           figsize=(24, 6),
                           layout='constrained')
    letters = iter('abcdefghijklmnopqrstuvwxyz')
    for i in range(len(ap_steps)):
        draw_two_layer(G_orig, ap_steps[i], layer_labels, i == 0, pos, ax[0, i])
        ax[0, i].text(-.4, .45, 1, fr'$\mathrm{{({next(letters)})}}$')
    fig.draw_without_rendering()
    fig.savefig(figtitle)

In [8]:
plt.rc('text', usetex=True)
plt.rc('font', size=22, **{'family': 'sans-serif', 'sans-serif': ['Helvetica']})
plt.rcParams['xtick.major.pad'] = 8
plt.rcParams['ytick.major.pad'] = 8

# Hyperbolic

In [11]:
N = 10
gamma = 3.
c = 10.
T = .4
nu = 0.
g = 0.

In [10]:
fig, (ax1, ax2) = plt.subplots(1, 2, subplot_kw=dict(projection='polar'))
plot_multiplex_with_ap_removal(N, gamma, c, T, nu, g, ax1, ax2, seed=42)
fig.savefig(
    'snapshot.pdf',
    bbox_inches='tight',
)

number of links:	 369
number of links:	 357

Starting AP removal
AP removal: iteration 1
AP removal: 3 APs
AP removal: iteration 2
AP removal: 0 APs


# Generated network AP removal plot

In [50]:
G = [nx.read_edgelist(f'links.coords{i}.txt', data=False) for i in {1, 2}]
G[0].add_nodes_from([n for n in G[1]])
G[1].add_nodes_from([n for n in G[0]])

In [51]:
%matplotlib inline

In [52]:
# pos = nx.kamada_kawai_layout(G[0])
pos = nx.spring_layout(G[0])

In [53]:
draw_process(
    G, pos,
    [
        '1',
        '2',
    ],
    # 'figs/toy_example/toy_example',
    # 'figs/toy_example/toy_example_1',
    'example',
)

KeyboardInterrupt: 